In [17]:
import torch
from torch import nn
import torchvision
import numpy as np

batch_size = 256
num_workers= 4
mnist_train = torchvision.datasets.FashionMNIST(root='Datasets',
                                    train = True, download=True, transform=torchvision.transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='Datasets',
                                    train=False,download=True,transform=torchvision.transforms.ToTensor())

train_iter = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,shuffle=True,num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,shuffle=True,num_workers=num_workers)

num_inputs = 784
num_hiddens = 256
num_outputs = 10


In [18]:
class FlattenLayer(nn.Module):
    def __init_(self):
        super(FlattenLayer,self).__init__()
    def forward(self,x):
        return x.view(x.shape[0],-1)
net = nn.Sequential(
         FlattenLayer(),
         nn.Linear(num_inputs,num_hiddens),
         nn.ReLU(),
         nn.Linear(num_hiddens,num_outputs))

print(net,net.parameters())
#for params in net.parameters():
#    print(params.size())
#    nn.init.normal_(params,mean=0,std=0.01)  #为了演示用，pytorch针对不同类型的layer会有比较合理的不同的初始化方式

def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n    

Sequential(
  (0): FlattenLayer()
  (1): Linear(in_features=784, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=10, bias=True)
) <generator object Module.parameters at 0x7f5060120450>


In [19]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr=0.5)

#optimizer_w = torch.optim.SGD(params=[net.weight], lr=lr, weight_decay=wd)#对权重偏差衰减
#optimizer_b = torch.optim.SGD(params=[net.bias], lr=lr) #bias不衰减   3.12加入的 #下面zero_grad和step分别写出

num_epochs =5

for epoch in range(num_epochs):
    train_l_sum,train_acc_sum,n = 0.0,0.0,0
    for X,y in train_iter:
        y_hat = net(X)
        l = loss(y_hat,y).sum()
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        
        train_l_sum += l
        train_acc_sum +=(y_hat.argmax(dim=1)==y).sum().item()
        n +=y.shape[0]
    test_acc = evaluate_accuracy(test_iter, net)
    print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n,test_acc))
        

epoch 1, loss 0.0030, train acc 0.720, test acc 0.771
epoch 2, loss 0.0018, train acc 0.826, test acc 0.802
epoch 3, loss 0.0016, train acc 0.848, test acc 0.771
epoch 4, loss 0.0015, train acc 0.859, test acc 0.838
epoch 5, loss 0.0014, train acc 0.865, test acc 0.827
